In [24]:
import torch

In [25]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros(X.shape[0]-h+1, X.shape[1]-w+1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = torch.sum(X[i:i+h, j:j+w] * K)
    return Y
def corr2d_multi_in(X, K):
    return sum(corr2d(x, k) for x, k in zip(X, K))

In [26]:
X = torch.randn(2, 3, 3)
K = torch.randn(2, 2, 2)

corr2d_multi_in(X, K)

tensor([[  6.2529,  -7.4813],
        [-10.3986,   4.7636]])

In [27]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], dim=0)

In [28]:
K = torch.stack((K, K+1, K+2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [29]:
corr2d_multi_in_out(X, K)

tensor([[[  6.2529,  -7.4813],
         [-10.3986,   4.7636]],

        [[  5.0921,  -6.9369],
         [ -8.6338,   2.0690]],

        [[  3.9314,  -6.3925],
         [ -6.8689,  -0.6256]]])

## 1x1卷积层

In [30]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h*w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [31]:
X = torch.normal(0, 1, size=(3, 3, 3))
K = torch.normal(0, 1, size=(2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

## Exercises

1. 假设我们有两个卷积核，大小分别为$k_1$和$k_2$(中间没有非线性激活函数)
   - 证明运算可以用单次卷积来表示
   - 这个等效的单个卷积核的维数是多少呢？
   - 反之亦然吗？

2. 假设输入为$(c_i, h, w)$，卷积核大小为$(c_o, c_i, k_h, k_w)$，填充为$(p_h, p_w)$，步幅为$(s_h, s_w)$
   - 前向传播的计算成本（乘法和加法）是多少？
   - 内存占用是多少？
   - 反向传播的内存占用是多少？
   - 反向传播的计算成本是多少？

3. 如果我们将输入通道$c_i$和输出通道$c_o$的数量加倍，计算数量会增加多少？如果我们把填充数量翻一番会怎么样？

4. 如果卷积核的高度和宽度是$k_h = k_w = 1$，前向传播的计算复杂度是多少？

5. 本节最后一个示例中的变量Y1和Y2是否完全相同？为什么？

6. 当卷积窗口不是$1*1$时，如何使用矩阵乘法实现卷积？